0. Setup

In [8]:
# import opencvs sift
import cv2
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import codecs
import pandas as pd
import seaborn as sns
import torch
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import sklearn as sk
from tqdm.notebook import tqdm



device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Set seed reproducible results
SEED = 42
np.random.seed(SEED)

1. Loading Data

In [9]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    path = '/content/drive/MyDrive/0Personal/Denmark/Job/Hackathon/settlersofagar/data/'

except:
    IN_COLAB = False
    path = "data/"

#files_ = os.listdir(path)
#files_jpg = [x for x in files_ if 'jpg' in x]
#files_json = [x for x in files_ if 'json' in x]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import zipfile
from PIL import Image

imgzip = zipfile.ZipFile(path + "train_data.zip")
inflist = imgzip.infolist()

# load image
images = {}
jsons = {}

for f in tqdm(inflist):
  if 'jpg' in f.filename:
    ifile = imgzip.open(f)
    images[f.filename] = Image.open(ifile)

  0%|          | 0/21582 [00:00<?, ?it/s]

In [13]:
for f in tqdm(inflist):
  if 'json' in f.filename:
    ifile = imgzip.open(f)
    jsons[f.filename] = json.load(ifile)

  0%|          | 0/21582 [00:00<?, ?it/s]

In [19]:
len(images)

10791

In [18]:
len(jsons)

10791

In [22]:
jsons.keys()

dict_keys(['100.json', '1000.json', '1001.json', '1003.json', '1004.json', '1008.json', '1009.json', '101.json', '1010.json', '1011.json', '1012.json', '1013.json', '1014.json', '1015.json', '1017.json', '1019.json', '102.json', '1020.json', '1022.json', '1023.json', '1024.json', '1027.json', '1028.json', '1029.json', '103.json', '1030.json', '1031.json', '1032.json', '1034.json', '1035.json', '1036.json', '1037.json', '1038.json', '1039.json', '104.json', '1040.json', '1041.json', '1042.json', '1043.json', '1044.json', '1045.json', '1046.json', '1047.json', '1048.json', '1049.json', '105.json', '1050.json', '1051.json', '1052.json', '1053.json', '1054.json', '1055.json', '1056.json', '1057.json', '1058.json', '106.json', '1061.json', '1062.json', '1064.json', '1065.json', '1066.json', '1067.json', '1068.json', '1069.json', '107.json', '1070.json', '1071.json', '1072.json', '1073.json', '1074.json', '1075.json', '1077.json', '1078.json', '1079.json', '1080.json', '1081.json', '1082.jso

In [23]:
jsons['880.json']

{'background': 'bright',
 'classes': ['S.aureus'],
 'colonies_number': 6,
 'labels': [{'class': 'S.aureus',
   'height': 37,
   'id': 1,
   'width': 37,
   'x': 1436,
   'y': 1041},
  {'class': 'S.aureus',
   'height': 45,
   'id': 2,
   'width': 45,
   'x': 1027,
   'y': 2569},
  {'class': 'S.aureus',
   'height': 45,
   'id': 3,
   'width': 45,
   'x': 846,
   'y': 2755},
  {'class': 'S.aureus',
   'height': 45,
   'id': 4,
   'width': 45,
   'x': 1773,
   'y': 2857},
  {'class': 'S.aureus',
   'height': 41,
   'id': 5,
   'width': 41,
   'x': 2542,
   'y': 2179},
  {'class': 'S.aureus',
   'height': 45,
   'id': 6,
   'width': 45,
   'x': 2619,
   'y': 2188}],
 'sample_id': 881}

In [26]:
df_image_description = pd.DataFrame(jsons).T.reset_index().rename(columns={'index':'id_image'})
df_image_description['colonies_number'].value_counts(normalize=True).reset_index().sort_values('index')

,index,colonies_number
0,0,0.090260
1,1,0.074136
2,2,0.073116
3,3,0.059494
4,4,0.050320
...,...,...
216,289,0.000093
220,294,0.000093
187,297,0.000185
250,299,0.000093


### Preprocessing

## 1. Circle Area